# Introduction to `pandas`

:::{note}
This material is mostly adapted from the following resources:
- [Earth and Environmental Data Science: Pandas](https://earth-env-data-science.github.io/lectures/pandas/pandas_intro.html)
- [Python Programming for Data Science: Pandas](https://www.tomasbeuzen.com/python-programming-for-data-science/chapters/chapter7-pandas.html)
:::

<img src="https://pandas.pydata.org/static/img/pandas.svg" width="300px" />

[Pandas](http://pandas.pydata.org/) is a an open source library providing Excel-like tables in Python.
It offers functionality for efficiently reading, writing, and processing data such as sorting, filtering, aggregating, and visualizing. Moreover, it provides tools for handling missing data and time series data.

<img src="https://media.geeksforgeeks.org/wp-content/cdn-uploads/creating_dataframe1.png" width="720px" />

:::{note}
Documentation for this package is available at https://pandas.pydata.org/docs/.
:::

:::{note}
If you have not yet set up Python on your computer, you can execute this tutorial in your browser via [Google Colab](https://colab.research.google.com/). Click on the rocket in the top right corner and launch "Colab". If that doesn't work download the `.ipynb` file and import it in [Google Colab](https://colab.research.google.com/).

Then install `pandas` and `numpy` by executing the following command in a Jupyter cell at the top of the notebook.

```sh
!pip install pandas numpy
```
:::

In [ ]:
import pandas as pd
import numpy as np

## The `Series`

A Series represents a one-dimensional array of data. The main difference between a Series and numpy array is that a Series has an **index**. The index contains the labels that we use to access the data. It is actually quite similar to a Python dictionary, where each value is associated with a key.

There are many ways to [create a Series](https://pandas.pydata.org/pandas-docs/stable/dsintro.html#series), but the core constructor is [`pd.Series()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.html) which can process a dictionary to create a Series.

:::{note}
The data used below is from Wikipedia's [List of power stations in Germany](https://en.wikipedia.org/wiki/List_of_power_stations_in_Germany#Nuclear).
:::

In [ ]:
dictionary = {
    "Neckarwestheim": 1269,
    "Isar 2": 1365,
    "Emsland": 1290,
}
s = pd.Series(dictionary)
s

In [ ]:
dictionary

Arithmetic operations and most `numpy` functions can be applied to `pd.Series`.
An important point is that the Series keep their index during such operations.

In [ ]:
np.log(s) / s**0.5

We can access the underlying index object if we need to:

In [ ]:
s.index

We can get values back out using the index via the `.loc` attribute

In [ ]:
s.loc["Isar 2"]

Or by raw position using `.iloc`

In [ ]:
s.iloc[2]

We can pass a list or array to loc to get multiple rows back:

In [ ]:
s.loc[["Neckarwestheim", "Emsland"]]

And we can even use so-called *slicing* notation (`:`) to get ranges of rows:

In [ ]:
s.loc["Neckarwestheim":"Emsland"]

In [ ]:
s.iloc[:2]

If we need to, we can always get the raw data back out as well

In [ ]:
type(s.values)  # a numpy array

## The `DataFrame`

There is a lot more to a `pandas.Series`, but they are limit to a single **column**. A more broadly useful Pandas data structure is the **DataFrame**. `pandas.DataFrame` is a collection of series that share the same index. It's a lot like a table in a spreadsheet.

The core constructor is `pd.DataFrame()`, which can be used like this using a dictionary of lists:

In [ ]:
data = {
    "capacity": [1269, 1365, 1290],  # MW
    "type": ["PWR", "PWR", "PWR"],
    "start_year": [1989, 1988, 1988],
    "end_year": [np.nan, np.nan, np.nan],
}

In [ ]:
df = pd.DataFrame(data, index=["Neckarwestheim", "Isar 2", "Emsland"])
df

We can also switch columns and rows very easily using the `.T` (transpose) attribute:

In [ ]:
df

A wide range of statistical functions are available on both Series and DataFrames.

In [ ]:
df.min()

In [ ]:
df.mean(numeric_only=True)

In [ ]:
df.describe()

We can get a single column as a Series using python's getitem syntax on the DataFrame object.

In [ ]:
df["capacity"]

...or using attribute syntax.

In [ ]:
df.end_year

Indexing works very similar to series

In [ ]:
df.loc["Emsland"]

In [ ]:
df.iloc[2]

But we can also specify the column(s) and row(s) we want to access

In [ ]:
df.loc["Emsland", "start_year"]

In [ ]:
df.loc[["Emsland", "Neckarwestheim"], ["start_year", "end_year"]]

Mathematical operations work as well, either on the whole DataFrame or on specific columns, the result of which can be assigned to a new column:

In [ ]:
df.capacity * 0.8

In [ ]:
df["reduced_capacity"] = df.capacity * 0.8
df

## Cleaning Data

We can also **remove** columns or rows from a DataFrame:

In [ ]:
df.drop("reduced_capacity", axis="columns")
df

We can update the variable `df` by either overwriting `df` or passing an `inplace` keyword:

In [ ]:
df = df.drop("reduced_capacity", axis="columns")

In [ ]:
df

We can also **drop columns** with only NaN values

In [ ]:
df.dropna(axis=1, how="any")

Or fill it up with default **fallback** data:

In [ ]:
df.fillna(2023)
df

Say, we already have one value for `end_year` and want to fill up the **missing data**. We can use forward fill (`ffill`) or backward fill (`bfill`):

In [ ]:
df.loc["Emsland", "end_year"] = 2023
df.loc["Neckarwestheim", "end_year"] = 2026
df

In [ ]:
df["end_year"] = df["end_year"].ffill()
df
df["x"] = np.nan
df["y"] = np.nan
df

Sometimes it can be useful to **rename** columns:

In [ ]:
df.rename(columns=dict(x="lat", y="lon"))

Sometimes it can be useful to **replace** values:

In [ ]:
df.replace({"PWR": "Pressurized water reactor"})

In [ ]:
df

In many cases, we want to **modify** values in a dataframe based on some rule. To modify values, we need to use `.loc` or `.iloc`. It can be use to set a specific value or a set of values based on their index and column labels:

In [ ]:
df.loc["Isar 2", "start_year"] = 2000
df.loc["Emsland", "capacity"] += 10
df

It can even be a completely new column:

In [ ]:
operational = ["Neckarwestheim", "Isar 2", "Emsland"]
df.loc[operational, "y"] = [49.04, 48.61, 52.47]
df

## Combining Datasets

Pandas supports a wide range of methods for merging different datasets. These are described extensively in the [documentation](https://pandas.pydata.org/pandas-docs/stable/merging.html). Here we just give a few examples.

In [ ]:
data = {
    "capacity": [1288, 1360, 1326],  # MW
    "type": ["BWR", "PWR", "PWR"],
    "start_year": [1985, 1985, 1986],
    "end_year": [2021, 2021, 2021],
    "x": [10.40, 9.41, 9.35],
    "y": [48.51, 52.03, 53.85],
}
df2 = pd.DataFrame(data, index=["Gundremmingen", "Grohnde", "Brokdorf"])
df2

We can now add this additional data to the `df` object

In [ ]:
df = pd.concat([df, df2])
df

## Sorting & Filtering Data

We can also **sort** the entries in dataframes, e.g. alphabetically by index or numerically by column values

In [ ]:
df.sort_index()

In [ ]:
df.sort_values(by="end_year", ascending=False)

We can also **filter** a DataFrame using a boolean series obtained from a condition. This is very useful to build subsets of the DataFrame.

In [ ]:
df.capacity > 1300

In [ ]:
df[df.capacity > 1300]

We can also combine multiple conditions, but we need to wrap the conditions with brackets!

In [ ]:
df[(df.capacity > 1300) & (df.start_year >= 1988)]

Or we make [SQL-like](https://en.wikipedia.org/wiki/SQL) queries:

In [ ]:
df.query("start_year == 1988")

In [ ]:
threshold = 1300
df.query("start_year == 1988 and capacity > @threshold")

In [ ]:
df

## Applying Functions

Sometimes it can be useful to **apply** a function to all values of a column/row. For instance, we might be interested in normalised capacities relative to the largest nuclear power plant:

In [ ]:
def normalise(s):
    return s / df.capacity.max()


df.capacity.apply(normalise)

For simple functions, there's often an easier alternative:

In [ ]:
def normalise(s: float):
    # adsfjielfwa
    return s / df.capacity.max()


df.capacity.apply(normalise)

But the `.apply()` function often gives you more flexibility.

## Plotting

DataFrames have all kinds of [useful plotting](https://pandas.pydata.org/pandas-docs/stable/visualization.html) built in.

:::{note}
Note, that we do not even have to import `matplotlib` for this. It is used under the hood by `pandas` to create the plots.
:::

In [ ]:
df.plot(kind="scatter", x="start_year", y="capacity")

In [ ]:
df.capacity.plot.barh(color="orange")
import matplotlib.pyplot as plt

plt.xlabel("Capacity (MW)")

## Time Indexes

Indexes are very powerful. They are a big part of why Pandas is so useful. There are different indices for different types of data. Time Indexes are especially great when handling time-dependent data.

In [ ]:
time = pd.date_range(start="2021-01-01", end="2023-01-01", freq="D")
time.dayofyear

In [ ]:
values = np.sin(2 * np.pi * time.dayofyear / 365)
values

In [ ]:
values = np.sin(2 * np.pi * time.dayofyear / 365)
ts = pd.Series(values, index=time)
ts

In [ ]:
ts.plot()

We can use Python's _slicing_ notation inside `.loc` to select a date range.

In [ ]:
ts.loc["2021-01-01":"2021-07-01"].plot()

In [ ]:
ts.loc["2021-05"].plot()

The `pd.TimeIndex` object has lots of useful attributes

In [ ]:
ts.index.month

In [ ]:
ts.index.weekday

Another common operation is to **change the resolution** of a dataset by resampling in time. Pandas exposes this through the `.resample()` function. The resample periods are specified using pandas [offset index](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases) syntax.

Below, we **resample** the dataset by taking the mean over each month.

In [ ]:
ts.resample("12h").mean().interpolate()

In [ ]:
ts.resample("ME").mean().plot()

## Reading and Writing Files

To **read** data into pandas, we can use for instance the [`pd.read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) function. This function is quite powerful and complex with many different settings. You can use it to extract data from almost any text file.

The `pd.read_csv()` function can take a path to a local file as an input, or even a hyperlink to an online text file.

Let's import a slightly larger dataset about the power plant fleet in Europe_

In [ ]:
fn = "https://raw.githubusercontent.com/PyPSA/powerplantmatching/master/powerplants.csv"
# fn = "powerplants(4).csv"

In [ ]:
df = pd.read_csv(fn, index_col=0)

In [ ]:
df = pd.read_csv(fn, index_col=0)
df.iloc[:5, :10]

In [ ]:
df.info()

In [ ]:
df.describe()

Sometimes, we also want to **store** a DataFrame for later use. There are many different file formats tabular data can be stored in,  including HTML, JSON, Excel, Parquet, Feather, etc. Here, let's say we want to store the DataFrame as CSV (comma-separated values) file under the name "tmp.csv".

In [ ]:
df.to_csv("tmp.csv")

In [ ]:
df

## Grouping and Aggregation

Both `Series` and `DataFrame` objects have a `groupby` method, which allows you to **group and aggregate** the data based on the values of one or more columns.

It accepts a variety of arguments, but the simplest way to think about it is that you pass another series, whose unique values are used to split the original object into different groups.

Here's an example which retrieves the total generation capacity per country.

In [ ]:
grouped = df.groupby("Country").Capacity.sum()
grouped.sort_values(ascending=False).head(10)

Such **chaining** of multiple operations is very common with pandas.

Let's break apart this operation a bit. The workflow with `groupby` can be divided into three general steps:

1. **Split**: Partition the data into different groups based on some criterion.
1. **Apply**: Do some calculation (e.g. aggregation or transformation) within each group.
1. **Combine**: Put the results back together into a single object.

<img src="https://miro.medium.com/max/1840/1*JbF6nhrQsn4f-TaSF6IR9g.png" width="720px" />

Grouping is not only possible on a single columns, but also on multiple columns. For instance,
we might want to group the capacities by country **and** fuel type. To achieve this, we pass a list of functions to the `groupby` functions.

In [ ]:
capacities = df.groupby(["Country", "Fueltype"]).Capacity.sum()
capacities

By grouping by multiple attributes, our index becomes a `pd.MultiIndex` (a hierarchical index with multiple *levels*.

In [ ]:
capacities.index[:5]

In [ ]:
type(capacities.index)

In [ ]:
df.nsmallest(10, "Capacity")

We can use the `.unstack` function to reshape the multi-indexed `pd.Series` into a `pd.DataFrame` which has the second index level as columns. 

In [ ]:
capacities.unstack().fillna(0.0).T.round(1).to_csv(
    "capacities_by_country_and_fueltype.csv"
)

**In summary,** the typical workflow with pandas consists of reading data from files, inspecting and cleaning the data, performing analysis through transformation and aggregation, visualizing the results, and storing the processed data for later use.

## Exercises

### Power Plants Data

In this exercise, we will use the [powerplants.csv](https://raw.githubusercontent.com/PyPSA/powerplantmatching/master/powerplants.csv) dataset from the [powerplantmatching](https://github.com/PyPSA/powerplantmatching) project. This dataset contains information about various power plants, including their names, countries, fuel types, capacities, and more.

URL: `https://raw.githubusercontent.com/PyPSA/powerplantmatching/master/powerplants.csv`

**Task 1:** Load the dataset into a pandas DataFrame.

**Task 2:** Run the function `.describe()` on the DataFrame.

**Task 3:** Provide a list of unique fuel types and technologies included in the dataset.

:::{note}
Look in the `pandas` documentation for functions that might be useful to solve these tasks.
:::

**Task 4:** Filter the dataset by power plants with the fuel type "Hard Coal".

**Task 5:** Identify the 5 largest coal power plants. In which countries are they located? When were they built?

**Task 6:** Identify the power plant with the longest name.

**Task 7:** Identify the 10 northernmost powerplants. What type of power plants are they?

**Task 8:** What is the average start year of each fuel type? Sort the fuel types by their average start year in ascending order and round to the nearest integer.

### Wind and Solar Capacity Factors

In this exercise, we will work with a time series dataset containing hourly wind and solar capacity factors for Ireland, taken from [model.energy](https://model.energy).

**Task 1:** Use `pd.read_csv` to load the dataset from the following URL into a pandas DataFrame. Ensure that the time stamps are treated as `pd.DatetimeIndex`.

**Task 2:** Calculate the mean capacity factor for wind and solar over the entire time period.



**Task 3:** Calculate the correlation between wind and solar capacity factors.

:::{note}
Go to the [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/index.html) for functions that might be useful to solve these tasks.
:::

**Task 4:** Plot the wind and solar capacity factors for the month of May.

**Task 5:** Plot the weekly average capacity factors for wind and solar over the entire time period.

**Task 6:** Go to [model.energy](https://model.energy) and retrieve the time series for another region of your choice. Recreate the analysis above and compare the results.

:::{note}
Look for "Download Comma-Separated-Variable (CSV) file of data" in Step 2.
:::